### Inspired By
https://github.com/RJBrooker/Q-learning-demo-Cartpole-V1/blob/master/cartpole.ipynb

In [2]:
from sklearn.preprocessing import KBinsDiscretizer
import numpy as np 
import time, math, random
from typing import Tuple
import tqdm
import gymnasium as gym

In [3]:
env = gym.make('CartPole-v1', render_mode='human')

In [4]:
n_bins = (20, 20, 20)
lower_bounds = [env.observation_space.low[0], env.observation_space.low[2], -math.radians(50)]
upper_bounds = [env.observation_space.high[0], env.observation_space.high[2], math.radians(50)]

def discretizer(location, _, angle, pole_velocity) -> Tuple[int, ...]:
    """Convert continuous state into a discrete state"""
    est = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='uniform')
    est.fit([lower_bounds, upper_bounds])
    return tuple(map(int, est.transform([[location, angle, pole_velocity]])[0]))


Initialise the Q value table with zeros. 

In [5]:
Q_table = np.zeros(n_bins + (env.action_space.n,))
Q_table.shape

(20, 20, 20, 2)

In [6]:
def policy( state : tuple ):
    """Choosing action based on epsilon-greedy policy"""
    return np.argmax(Q_table[state])


Update function 

In [7]:
def new_Q_value( reward : float ,  new_state : tuple , discount_factor=0.9 ) -> float:
    """Temperal diffrence for updating Q-value of state-action pair"""
    future_optimal_value = np.max(Q_table[new_state])
    learned_value = reward + discount_factor * future_optimal_value
    return learned_value

Decaying learning rate

In [8]:
# Adaptive learning of Learning Rate
def learning_rate(n : int , min_rate=0.01 ) -> float  :
    """Decaying learning rate"""
    return max(min_rate, min(1.0, 1.0 - math.log10((n + 1) / 25)))

Decaying exploration rate

In [9]:
def exploration_rate(n : int, min_rate= 0.01 ) -> float :
    """Decaying exploration rate"""
    # return max(min_rate, min(1, 1.0 - math.log10((n  + 1) / 25)))
    return max(min_rate, min(1, 1/np.log(n+2)))

## Training

In [10]:

n_episodes = 1000
for e in tqdm.tqdm(range(n_episodes)):
    
    current_state, done = discretizer(*env.reset()[0]), False
    steps = 0
    while done == False:
        steps = steps + 1
        if steps > 1000:
            break
        
        # policy action 
        action = policy(current_state) # exploit
        
        # insert random action
        if np.random.random() < exploration_rate(e) : 
            action = env.action_space.sample() # explore 
         
        # increment enviroment
        obs, reward, done, _, extra = env.step(action)
        new_state = discretizer(*obs)
        
        # Update Q-Table
        lr = learning_rate(e)
        learnt_value = new_Q_value(reward, new_state)
        old_value = Q_table[current_state][action]
        Q_table[current_state][action] = (1-lr)*old_value + lr*learnt_value
        
        current_state = new_state
        
        # Render the cartpole environment
        env.render()
            
Q_table
            
            

  0%|          | 1/1000 [00:04<1:13:49,  4.43s/it]

36


  0%|          | 2/1000 [00:05<43:54,  2.64s/it]  

33


  0%|          | 3/1000 [00:07<33:04,  1.99s/it]

...

28


 24%|██▍       | 242/1000 [03:45<10:55,  1.16it/s]

17


 24%|██▍       | 243/1000 [03:46<10:51,  1.16it/s]

20


In [42]:
current_state, done = discretizer(*env.reset()[0]), False
steps = 0
while done==False:

    steps = steps + 1
    if steps > 1000:
        break
    
    action = policy(current_state) # exploit
    obs, reward, done, _, extra = env.step(action)
    new_state = discretizer(*obs)
    
    learnt_value = new_Q_value(reward, new_state)
    
    current_state = new_state
    
    env.render()
print (steps)

675
